In [ ]:
import os, json
import pandas as pd
import numpy as np

In [ ]:
ais1 = pd.read_csv('/Users/wangpeijia/Desktop/research/Hist_AIS_data_1.csv')
ais2 = pd.concat([pd.read_csv('/Users/wangpeijia/Desktop/research/Hist_AIS_data_2.csv'), ais1])
ais = pd.concat([pd.read_csv('/Users/wangpeijia/Desktop/research/Hist_AIS_data_3.csv'), ais2])

In [ ]:
from geopy.distance import geodesic
offdist = (44.2284, -124.57144)
slfdist = (44.38132, -124.18216)
ais['offdist'] = [geodesic((a, b), offdist).miles for a, b in ais[['LAT', 'LON']].values]
ais['slfdist'] = [geodesic((a, b), slfdist).miles for a, b in ais[['LAT', 'LON']].values]

In [ ]:
ais['TIMESTAMP UTC'] = pd.to_datetime(ais['TIMESTAMP UTC']).dt.strftime('%Y-%m-%d %H:%M:%S')
ais['TIMESTAMP UTC'] = pd.to_datetime(ais['TIMESTAMP UTC'])

In [ ]:
import plotly.express as px
import datetime
def animate_map(starttime, endtime):
    subais = ais[(ais['TIMESTAMP UTC'] > starttime) & (ais['TIMESTAMP UTC'] < endtime)]
    time = subais['TIMESTAMP UTC']
    for i in time:
        subais.loc[len(subais.index)] = [0,0,'shelf hydrophone'+str(i),'shelf hydrophone',0,0,0, -124.18216,44.38132,0,0, i, 0,0]
        subais.loc[len(subais.index)] = [0,0,'offshore hydrophone'+str(i),'offshore hydrophone',0,0,10, -124.57144,44.2284,0,0, i, 0,0]
    subais = subais.reset_index()
    for i in range(len(subais)):
        subais.loc[i,'TIMESTAMP UTC'] = subais.iloc[i]['TIMESTAMP UTC']- datetime.timedelta(minutes=subais.iloc[i]['TIMESTAMP UTC'].minute % 10)
    subais['TIMESTAMP UTC'] = subais['TIMESTAMP UTC'].dt.to_period('T')
    fig = px.scatter_mapbox(subais,
              lat="LAT" ,
              lon="LON",
              hover_name="VESSEL TYPE",
              hover_data= ['SPEED (KNOTSx10)','slfdist'],
              color="VESSEL NAME",
              animation_frame=subais['TIMESTAMP UTC'].astype(dtype=str),
              mapbox_style='carto-positron',                 
              zoom=5)
    fig.show();
animate_map('2016-10-31 05:50:00', '2016-10-31 06:50:00')

ValueError: cannot set a row with mismatched columns